In [1]:
import numpy as np
import pufferlib, pufferlib.vector
from pufferlib.environments import classic_control

In [2]:
num_envs = 12

In [3]:
vecenv = pufferlib.vector.make(
    classic_control.env_creator("CartPole-v1"),
    num_envs=num_envs,
    backend=pufferlib.vector.Multiprocessing,
)

Process Process-7:
Process Process-11:
Process Process-4:
Process Process-1:
Process Process-6:
Process Process-12:
Process Process-2:
Process Process-10:
Process Process-5:
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/multiprocessing/process.py", line 314

In [4]:
env_rewards = [list() for _ in range(num_envs)]
env_rewards

returns = []

from itertools import count

dones = np.array([False] * num_envs)
truncateds = np.array([False] * num_envs)
dones, truncateds

obs, _ = vecenv.reset()
episodes = 0
for t in count():
    obs, rewards, dones, truncateds, _ = vecenv.step(vecenv.action_space.sample())
    for i, reward in enumerate(rewards):
        env_rewards[i].append(reward)
    for i in np.where(dones | truncateds)[0]:
        returns.append(sum(env_rewards[i]))
        env_rewards[i] = []
        episodes += 1
    if episodes >= 1_000:
        break

np.mean([np.sum(rets) for rets in returns]), t

(22.304, 1950)

In [5]:
import torch
from torch import nn

In [6]:
class QNetwork(nn.Module):
    def __init__(self, n_input, n_hiddens, n_actions):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(n_input, n_hiddens),
            nn.ReLU(),
            nn.Linear(n_hiddens, n_actions),
        )

    def forward(self, x):
        device = next(self.mlp.parameters()).device
        x = torch.tensor(np.array(x), dtype=torch.float32, device=device)
        
        return self.mlp(x)

In [7]:
import random

In [8]:
def select_actions(q_values, epsilon):
    batch_size, n_actions = q_values.shape
    if random.random() < epsilon:
        return np.random.choice(range(n_actions), batch_size)
    return np.array(q_values.argmax(dim=-1).cpu())

In [9]:
from collections import deque

In [10]:
class ReplayBuffer:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.queue = deque(maxlen=capacity)

    def store(self, transition):
        self.queue.append(transition)

    def extend(self, transitions):
        for transition in transitions:
            self.store(transition)

    def sample(self, n_samples: int):
        return [random.choice(self.queue) for _ in range(n_samples)]

    def __len__(self):
        return len(self.queue)

    def __repr__(self):
        return self.queue.__repr__()

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from torch import optim

from itertools import count
from tqdm.auto import tqdm

In [13]:
def train(
    n_target_update_eps = 60,
    n_episodes = 8_000,
    update_batch_size = 128,
    learning_rate = 1e-4,
    epsilon_start = 1.0,
    epsilon_end = 0.0,
    epsilon_decay_percent = 0.9,
    leave_bar = True,
    suggestion_uuid = None,
):
    net = QNetwork(n_input, n_hiddens, n_actions).to(device)
    target_net = QNetwork(n_input, n_hiddens, n_actions).to(device)
    target_net.load_state_dict(net.state_dict())
    target_net.eval()
    
    env_rewards = [list() for _ in range(num_envs)]
    env_rewards
    returns = []
    
    gamma = 1.0
    buffer_size = 100_000
    epsilon_decay = epsilon_decay_percent * n_episodes
    epsilon_decay_rate = (epsilon_start - epsilon_end) / epsilon_decay
    epsilon = epsilon_start
    
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    loss_fn = nn.SmoothL1Loss()
    buffer = ReplayBuffer(capacity=buffer_size)
    
    obs, _ = vecenv.reset()
    obs = obs.copy()
    episodes = 0
    bar = tqdm(
        desc="Episodes",
        total=n_episodes,
        initial=0,
        leave=leave_bar,
    )
    
    debug = False
    debug_seen = False
    
    for t in count():
        with torch.no_grad():
            q_values = net(obs)
            actions = select_actions(q_values, epsilon)
    
        if debug and debug_seen:
            print("Ob seen:\t", obs[0])
            print("Action taken:\t", actions[0])
        
        next_obs, rewards, dones, truncateds, _ = vecenv.step(actions)
    
        if debug and debug_seen:
            print("Ob seen:\t", obs[0])
        buffer.extend(zip(obs.copy(), actions.copy(), rewards.copy(), next_obs.copy(), (dones | truncateds).copy()))
        obs = next_obs.copy()
    
        for i, reward in enumerate(rewards):
            env_rewards[i].append(reward)
    
        if len(buffer) >= update_batch_size:
            obs_s, actions_s, rewards_s, next_obs_s, dones_s = zip(*buffer.sample(update_batch_size))
            if debug:
                print("\nOb:\t\t", obs_s[0])
                print("Action:\t", actions_s[0])
                print("Reward:\t", rewards_s[0])
                print("Next ob:\t", next_obs_s[0])
                print("Done:\t\t", dones_s[0])
        
                print("\nBuffer length:", len(buffer))
                print("\nBuffer items:", buffer.queue[0])
    
            actions_t = torch.tensor(actions_s, device=device).unsqueeze(1)
            q_values_t = net(obs_s).gather(1, actions_t).squeeze(1)
            rewards_t = torch.tensor(rewards_s, device=device) / 500.
            dones_t = torch.tensor(dones_s, device=device)
            with torch.no_grad():
                target_q_values_t = target_net(next_obs_s).max(dim=1).values
    
            target = rewards_t + gamma * ~dones_t * target_q_values_t
            loss = loss_fn(target, q_values_t)
    
            if debug:
                print("Actions T:\t", actions_t[0])
                print("Qs T:\t\t", q_values_t[0])
                print("Target Qs:\t", target_q_values_t[0])
                print("TD Target:\t", target[0])
                with torch.no_grad():
                    td_error = target - q_values_t
                print("TD Error:\t", td_error[0])
    
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            if debug:
                with torch.no_grad():
                    q_values_t = net(obs_s).gather(1, actions_t).squeeze(1)
                    target_q_values_t = target_net(next_obs_s).max(dim=1).values
                    target = rewards_t + gamma * ~dones_t * target_q_values_t
                    td_error = target - q_values_t
                    
                print("Actions T:\t", actions_t[0])
                print("Qs T:\t\t", q_values_t[0])
                print("TD Target:\t", target[0])
                print("Target Qs:\t", target_q_values_t[0])
                print("TD Error:\t", td_error[0])
        
                break
            
        for i in np.where(dones | truncateds)[0]:
            returns.append(sum(env_rewards[i]))
            env_rewards[i] = []
            episodes += 1
            if epsilon >= epsilon_end:
                epsilon -= epsilon_decay_rate
            bar.update()
            
        if episodes >= n_episodes:
            bar.close()
            break
            #debug = True
    
        if episodes % n_target_update_eps == 0:
            target_net.load_state_dict(net.state_dict())
    
    mean_last_20 = np.mean([np.sum(rets) for rets in returns[-20:]])
    max_last_20 = np.max([np.sum(rets) for rets in returns[-20:]])
    return net, mean_last_20, max_last_20, t

In [14]:
from carbs import CARBS, CARBSParams, LinearSpace, LogSpace, LogitSpace, ObservationInParam, ParamDictType, Param
import time

In [15]:
n_input = vecenv.single_observation_space.shape[0]
n_actions = vecenv.single_action_space.n
n_hiddens = 16

n_target_update_eps = 60
n_episodes = 600
update_batch_size = 128
learning_rate = 1e-4
epsilon_start = 1.0
epsilon_end = 0.05
epsilon_decay_percent = 0.8

param_spaces = [
    Param(name="n_target_update_eps", space=LinearSpace(is_integer=True, scale=3, min=10, max=10_000), search_center=60),
    Param(name="n_episodes", space=LogSpace(is_integer=True, min=200, max=8_000), search_center=600),
    Param(name="update_batch_size", space=LogSpace(is_integer=True, min=2, max=2048), search_center=128),
    Param(name="learning_rate", space=LogSpace(scale=0.5), search_center=1e-4),
    Param(name="epsilon_start", space=LinearSpace(min=0.5, max=1.0), search_center=0.9),
    Param(name="epsilon_end", space=LinearSpace(min=0.0, max=0.5), search_center=0.01),
    Param(name="epsilon_decay_percent", space=LogitSpace(), search_center=0.9),
]

carbs_params = CARBSParams(
    is_wandb_logging_enabled=False,
)

carbs = CARBS(carbs_params, param_spaces)

best_return = -1.0
best_time = 1e37
best_suggestion = None
pefects = 0

for run in count(1):
    suggestion = carbs.suggest().suggestion
    
    start_time = time.time()
    net, mean_last_20, max_last_20, t = train(leave_bar=False, **suggestion)
    elapsed_time = time.time() - start_time

    if mean_last_20 > best_return or (mean_last_20 == best_return and elapsed_time < best_time):
        best_return = mean_last_20
        best_time = elapsed_time
        best_suggestion = suggestion
    
    print(f"Run {run:<3}   ", end="")
    print(f"Time: {elapsed_time:6.2f}s", end="\t")
    
    print(f"Mean ret: {mean_last_20:6.2f}, ", end="")
    print(f"Max ret: {max_last_20:6.2f}", end="\t")
    print(f"\tBest: {best_return:6.2f}, {best_time:6.2f}s", end="\t\r")
    
    obs_out = carbs.observe(ObservationInParam(input=suggestion, output=mean_last_20, cost=elapsed_time))

    if mean_last_20 >= 500.0:
        perfects += 1
        if perfects >= 10:
            break

2025-02-16 13:15:15.620 | INFO     | carbs.carbs:__init__:76 - Running CARBS with config CARBSParams(better_direction_sign=1, seed=0, num_random_samples=4, is_wandb_logging_enabled=False, wandb_params=WandbLoggingParams(project_name=None, group_name=None, run_name=None, run_id=None, is_suggestion_logged=True, is_observation_logged=True, is_search_space_logged=True, root_dir='/mnt/private'), checkpoint_dir='checkpoints/', s3_checkpoint_path='s3://int8/checkpoints', is_saved_on_every_observation=True, initial_search_radius=0.3, exploration_bias=1.0, num_candidates_for_suggestion_per_dim=100, resample_frequency=5, max_suggestion_cost=None, min_pareto_cost_fraction=0.2, is_pareto_group_selection_conservative=True, is_expected_improvement_pareto_value_clamped=True, is_expected_improvement_value_always_max=False, outstanding_suggestion_estimator=<OutstandingSuggestionEstimatorEnum.THOMPSON: 'THOMPSON'>)


Episodes:   0%|          | 0/388 [00:00<?, ?it/s]

Run 1     Time:   1.50s	Mean ret:  10.55, Max ret:  20.00		Best:  10.55,   1.50s	

Episodes:   0%|          | 0/716 [00:00<?, ?it/s]

Run 2     Time:   2.39s	Mean ret:  10.90, Max ret:  13.00		Best:  10.90,   2.39s	

...

Episodes:   0%|          | 0/3278 [00:00<?, ?it/s]

Run 650   Time:  17.02s	Mean ret:  81.45, Max ret: 119.00		Best: 479.05,  69.67s	

Episodes:   0%|          | 0/5169 [00:00<?, ?it/s]

Run 651   Time:  32.81s	Mean ret: 116.70, Max ret: 181.00		Best: 479.05,  69.67s	

KeyboardInterrupt: 

In [ ]:
from pprint import pprint
pprint(best_suggestion)

In [ ]:
net, mean_last_100, max_last_100, t = train(**best_suggestion)
mean_last_100, max_last_100, t

In [ ]:
import gymnasium as gym
import numpy as np
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
video_folder = f"./videos/DQN_scratch_CartPole_{timestamp}"

env = gym.make("CartPole-v1", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, video_folder)

epsilon = best_suggestion["epsilon_end"]

for _ in range(9):
    ob, _ = env.reset()
    ob = np.expand_dims(ob, 0)
    ret = 0
    done, truncated = False, False
    while not (done or truncated):
        with torch.no_grad():
            q_values = net(ob)
        actions = select_actions(q_values, epsilon)
        ob, reward, done, truncated, _ = env.step(actions[0])
        ob = np.expand_dims(ob, 0)
        ret += reward
    
    print(ret)

env.close()